In [1]:
import os
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [2]:
# Define a new Pydantic model with field descriptions and tailored for Twitter.
class TwitterUser(BaseModel):
    name: str = Field(description="Full name of the user.")
    handle: str = Field(description="Twitter handle of the user, without the '@'.")
    age: int = Field(description="Age of the user.")
    hobbies: List[str] = Field(description="List of hobbies of the user.")
    email: str = Field(description="Email address of the user.")
    bio: str = Field(description="Bio or short description about the user.")
    location: str = Field(description="Location or region where the user resides.")
    is_blue_badge: bool = Field(
        description="Boolean indicating if the user has a verified blue badge."
    )
    joined: str = Field(description="Date the user joined Twitter.")
    gender: str = Field(description="Gender of the user.")
    appearance: str = Field(description="Physical description of the user.")
    avatar_prompt: str = Field(
        description="Prompt for generating a photorealistic avatar image. The image should capture the essence of the user's appearance description, ideally in a setting that aligns with their interests or bio. Use professional equipment to ensure high quality and fine details."
    )
    banner_prompt: str = Field(
        description="Prompt for generating a banner image. This image should represent the user's hobbies, interests, or the essence of their bio. It should be high-resolution and captivating, suitable for a Twitter profile banner."
    )

In [3]:
# Instantiate the parser with the new model.
parser = PydanticOutputParser(pydantic_object=TwitterUser)

# Update the prompt to match the new query and desired format.
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(
            "answer the users question as best as possible.\n{format_instructions}\n{question}"
        )
    ],
    input_variables=["question"],
    partial_variables={
        "format_instructions": parser.get_format_instructions(),
    },
)

In [4]:
chat_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key='',
    max_tokens=1000
)

# Generate the input using the updated prompt.
user_query = (
    "Generate a detailed Twitter profile of a random realistic user with a diverse background, "
    "from any country in the world, original name, including prompts for images. Come up with "
    "real name, never use most popular placeholders like john smith and john doe."
)
_input = prompt.format_prompt(question=user_query)

output = chat_model(_input.to_messages())
parsed = parser.parse(output.content)
print(output.content)
print(parsed)

{
  "name": "Lila Patel",
  "handle": "lilapatel",
  "age": 30,
  "hobbies": ["Photography", "Cooking", "Hiking"],
  "email": "lila.patel@example.com",
  "bio": "Passionate about capturing moments through the lens. Foodie, nature lover, and adventurer. Living life one click at a time.",
  "location": "Mumbai, India",
  "is_blue_badge": false,
  "joined": "2015-07-12",
  "gender": "Female",
  "appearance": "Curly hair, dusky complexion, and expressive eyes.",
  "avatar_prompt": "Create a photorealistic avatar image showcasing my love for photography. Capture me holding a camera with a beautiful sunset in the background.",
  "banner_prompt": "Design a captivating banner image that represents my hobbies. Include elements of photography, cooking, and hiking in a scenic natural setting."
}
name='Lila Patel' handle='lilapatel' age=30 hobbies=['Photography', 'Cooking', 'Hiking'] email='lila.patel@example.com' bio='Passionate about capturing moments through the lens. Foodie, nature lover, and 

In [5]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("elon.pdf")
document = loader.load()

In [7]:
document_query = "Create a profile based on this description: " + document[0].page_content

_input = prompt.format_prompt(question=document_query)
output = chat_model(_input.to_messages())
parsed = parser.parse(output.content)

print(type(parsed))

<class '__main__.TwitterUser'>


In [9]:
from langchain.output_parsers import OutputFixingParser
from langchain.schema import OutputParserException

try:
    parsed = parser.parse(output.content)
except OutputParserException as e:
    new_parser = OutputFixingParser.from_llm(
        parser=parser,
        llm=ChatOpenAI()
    )
    parsed = new_parser.parse(output.content)
parsed

TwitterUser(name='Elon Musk', handle='elonmusk', age=51, hobbies=['space exploration', 'electric vehicles', 'artificial intelligence', 'sustainable energy', 'tunnel construction', 'neural interfaces', 'Mars colonization', 'hyperloop transportation'], email='', bio="Elon Musk, a 51-year-old male entrepreneur, inventor, and CEO, is best known for his ambitious goals in revolutionizing transportation and energy. Born in Pretoria, South Africa, Musk later moved to the United States to pursue higher education. He attended Queen's University in Kingston, Ontario, Canada for two years before transferring to the University of Pennsylvania. As a visionary with a normal build, short-cropped hair, and a trimmed beard, Musk often sports tailored suits or smart casual attire, giving him a confident yet approachable demeanor. Throughout his career, Musk has founded and led several successful companies, including SpaceX, Tesla, Neuralink, and The Boring Company. His interests span across various fiel